In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from keras import backend as K
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using TensorFlow backend.


In [27]:
#path variables
#game_url = "https://www.coolmathgames.com/0-worlds-hardest-game/play"

# use if offline, run these first
# cd C:\Users\mrchr\Documents\deep-rl-worlds-hardest-game
# python -m http.server 8080

game_url = "http://localhost:8080/game/" 

chrome_driver_path = "res/chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"

#scripts

#get image from canvas
getbase64Script = "canvasRunner = document.getElementById('twhgCanvas'); \
return canvasRunner.toDataURL().substring(22);"

In [3]:
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* get_crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* get_playing(): true if game in progress, false is crashed or paused
* press_up(): sends a single to press up get to the browser
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(810, 600)
        self._driver.get(game_url)
    def get_crashed(self):
        return self._driver.execute_script("return player.dying")
    def get_playing(self):
        return self._driver.execute_script("return state == 'game' && !player.dying")
    def get_completed(self):
        if self._driver.execute_script("return finishLevelTimer > 0"):
            self._driver.execute_script("finishLevelTimer = 1")
            return True
        return False
    def press_up(self, stride):
        start_y = self._driver.execute_script("return player.y")
        prev_y = -1
        curr_y = start_y
        self._driver.execute_script("keyUp = true")
        self._driver.execute_script("keydown.up = true")
        while curr_y >= start_y - stride:
            if curr_y == prev_y:
                self._driver.execute_script("keydown.up = false")
                self._driver.execute_script("keyUp = false")
                return True
            prev_y = curr_y
            time.sleep(0.025)
            curr_y = self._driver.execute_script("return player.y")
        self._driver.execute_script("keydown.up = false")
        self._driver.execute_script("keyUp = false")
        return False
    def press_down(self, stride):
        start_y = self._driver.execute_script("return player.y")
        prev_y = -1
        curr_y = start_y
        self._driver.execute_script("keyDown = true")
        self._driver.execute_script("keydown.down = true")
        while curr_y < start_y + stride:
            if curr_y == prev_y:
                self._driver.execute_script("keydown.down = false")
                self._driver.execute_script("keyDown = false")
                return True
            prev_y = curr_y
            time.sleep(0.025)
            curr_y = self._driver.execute_script("return player.y")
        self._driver.execute_script("keydown.down = false")
        self._driver.execute_script("keyDown = false")
        return False
    def press_left(self, stride):
        start_x = self._driver.execute_script("return player.x")
        prev_x = -1
        curr_x = start_x
        self._driver.execute_script("keyLeft = true")
        self._driver.execute_script("keydown.left = true")
        while curr_x >= start_x - stride:
            if curr_x == prev_x:
                self._driver.execute_script("keydown.left = false")
                self._driver.execute_script("keyLeft = false")
                return True
            prev_x = curr_x
            time.sleep(0.025)
            curr_x = self._driver.execute_script("return player.x")
        self._driver.execute_script("keydown.left = false")
        self._driver.execute_script("keyLeft = false")
        return False
    def press_right(self, stride): # 120ms to 50ms to move
        start_x = self._driver.execute_script("return player.x")
        prev_x = -1
        curr_x = start_x
        self._driver.execute_script("keyRight = true")
        self._driver.execute_script("keydown.right = true")
        while curr_x < start_x + stride:
            if curr_x == prev_x:
                self._driver.execute_script("keydown.right = false")
                self._driver.execute_script("keyRight = false")
                return True
            prev_x = curr_x
            time.sleep(0.025)
            curr_x = self._driver.execute_script("return player.x")
        self._driver.execute_script("keydown.right = false")
        self._driver.execute_script("keyRight = false")
        return False
    def pause(self):
        return self._driver.execute_script("paused = true")
    def resume(self):
        return self._driver.execute_script("paused = false")
    def end(self):
        self._driver.close()
    def enter_level(self):
        time.sleep(0.50) # wait for things to load
        while self._driver.execute_script("return state !== 'game'"):
            self._driver.find_element_by_id("twhgCanvas").click()

In [10]:
game = Game()
agent = Agent(game)
game_state = Game_sate(agent,game)
game.enter_level()

C:\Users\mrchr\Miniconda3\envs\tensorflow_gpuenv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [10]:
agent.up()
agent.down()
agent.left()
agent.right()

False

In [65]:

agent.left()
agent.left()

True

False

In [4]:
class Agent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game;
        self.stride = 16
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def won(self):
        return self._game.get_completed()
    def up(self):
        return self._game.press_up(self.stride)
    def down(self):
        return self._game.press_down(self.stride)
    def left(self):
        return self._game.press_left(self.stride)
    def right(self):
        return self._game.press_right(self.stride)

In [6]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine
    def get_block_map(self):
        screen = grab_screen(self._game._driver)
        hsv = cv2.cvtColor(screen, cv2.COLOR_BGR2HSV)
        block_map = np.zeros([img_cols, img_rows])

        # purplish outside
        mask1 = cv2.inRange(hsv, (95, 50, 50), (135, 255, 255))
        grey_img = cv2.cvtColor(cv2.bitwise_and(screen, screen, mask=mask1), cv2.COLOR_RGB2GRAY)

        for r in range(grey_img.shape[0]//scale):
            for c in range(grey_img.shape[1]//scale):
                if np.any(grey_img[r*scale:r*scale+scale, c*scale:c*scale+scale] >= 128):
                    block_map[r, c] = 2

        # red
        mask2 = cv2.inRange(hsv, (0, 50, 50), (18, 255, 255))
        mask2 = cv2.bitwise_or(mask2, cv2.inRange(hsv, (165, 50, 50), (180, 255, 255)))
        grey_img = cv2.cvtColor(cv2.bitwise_and(screen, screen, mask=mask2), cv2.COLOR_RGB2GRAY)

        for r in range(grey_img.shape[0]//2):
            for c in range(grey_img.shape[1]//2):
                if np.any(grey_img[r*scale:r*scale+scale, c*scale:c*scale+scale] > 0) and block_map[r, c] == 0:
                    block_map[r, c] = 1

        goal_location = self._game._driver.execute_script("return checkpoints[level][checkpoints[level].length - 1]")

        for c in range(goal_location[0] * scale, scale*(goal_location[0] + goal_location[2])):
            for r in range(scale*(goal_location[1] - 1), scale*(goal_location[1] - 1 + goal_location[3])):
                block_map[r, c] = 4

        return block_map
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = 1
        action_reward = -0.2
        is_over = False #game over
        if actions[1] == 1:
            if self._agent.up():
                action_reward = -1
            else:
                action_reward = -0.1
        elif actions[2] == 1:
            if self._agent.down():
                action_reward = -1
            else:
                action_reward = -0.1
        elif actions[3] == 1:
            if self._agent.left():
                action_reward = -1
            else:
                action_reward = -0.1
        elif actions[4] == 1:
            if self._agent.right():
                action_reward = -1
            else:
                action_reward = -0.1
        
        image = self.get_block_map()
        #image = grab_screen(self._game._driver)
        self._display.send(image) #display the image on screen
        
        if self._agent.is_crashed():
            action_reward = -100
            is_over = True
        elif self._agent.won():
            action_reward = 100
            print("WE JUST WON")
            is_over = True
        
        return image, action_reward , is_over #return the Experience tuple

In [7]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR to RGB
    image = image[50:550, :] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (re_img_rows, re_img_cols))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(window_title, 200, 130)
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [8]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [9]:
#game parameters
ACTIONS = 5 # possible actions: up, down, left, right, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 40, 26
re_img_rows, re_img_cols = 80, 52
scale = re_img_rows//img_rows
img_channels = 4 #We stack 4 frames

In [10]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [22]:
'''Call only once to init file structure
'''
init_cache()

In [19]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(16, (4, 4), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #100*63*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    #model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [13]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")


In [31]:
#main function
def playGame(observe=False):
    game = Game()
    agent = Agent(game)
    game_state = Game_sate(agent, game)
    game.enter_level()
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [32]:
playGame(observe=False);

TIMESTEP 66000 / STATE explore / EPSILON 0.034166898999899414 / ACTION 0 / REWARD -0.2 / Q_MAX  -16.492899 / Loss  0.05361112207174301
fps: 0.02086223018216734
TIMESTEP 66001 / STATE explore / EPSILON 0.03416589999989941 / ACTION 0 / REWARD -0.2 / Q_MAX  -15.619086 / Loss  0.04048635810613632
fps: 2.9648848486562143
TIMESTEP 66002 / STATE explore / EPSILON 0.03416490099989941 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.284043 / Loss  0.001581263728439808
fps: 3.0874955188920565
TIMESTEP 66003 / STATE explore / EPSILON 0.03416390199989941 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.286202 / Loss  0.004509373567998409
fps: 2.9027711357843384
TIMESTEP 66004 / STATE explore / EPSILON 0.03416290299989941 / ACTION 0 / REWARD -0.2 / Q_MAX  -19.82922 / Loss  0.07471456378698349
fps: 2.999134075316141
TIMESTEP 66005 / STATE explore / EPSILON 0.03416190399989941 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.836151 / Loss  0.008649476803839207
fps: 3.1560229468526333
TIMESTEP 66006 / STATE explore / EPSILON 0.034160

fps: 3.197696971175545
TIMESTEP 66052 / STATE explore / EPSILON 0.034114950999899335 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.342867 / Loss  0.02006625384092331
fps: 3.285431959123854
TIMESTEP 66053 / STATE explore / EPSILON 0.03411395199989933 / ACTION 0 / REWARD -0.2 / Q_MAX  -18.798283 / Loss  0.015891294926404953
fps: 3.322447311262391
TIMESTEP 66054 / STATE explore / EPSILON 0.03411295299989933 / ACTION 0 / REWARD -0.2 / Q_MAX  -18.732952 / Loss  0.0075697884894907475
fps: 3.0978209731061566
TIMESTEP 66055 / STATE explore / EPSILON 0.03411195399989933 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.35911 / Loss  0.04112837836146355
fps: 3.131271701372688
TIMESTEP 66056 / STATE explore / EPSILON 0.03411095499989933 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.897244 / Loss  0.011971576139330864
fps: 3.295294842302422
TIMESTEP 66057 / STATE explore / EPSILON 0.03410995599989933 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.49868 / Loss  0.059569183737039566
fps: 3.07901422894783
TIMESTEP 66058 / STATE explore 

fps: 2.6605922782445424
TIMESTEP 66104 / STATE explore / EPSILON 0.034063002999899256 / ACTION 3 / REWARD -1 / Q_MAX  -14.528608 / Loss  0.14363530278205872
fps: 2.605151291549586
TIMESTEP 66105 / STATE explore / EPSILON 0.034062003999899254 / ACTION 3 / REWARD -1 / Q_MAX  -6.8372946 / Loss  0.31401509046554565
fps: 2.5925957518879024
TIMESTEP 66106 / STATE explore / EPSILON 0.03406100499989925 / ACTION 3 / REWARD -1 / Q_MAX  -13.949948 / Loss  0.0656026303768158
fps: 2.6004817424568865
TIMESTEP 66107 / STATE explore / EPSILON 0.03406000599989925 / ACTION 3 / REWARD -1 / Q_MAX  -10.540489 / Loss  0.19461935758590698
fps: 2.5806448437297154
TIMESTEP 66108 / STATE explore / EPSILON 0.03405900699989925 / ACTION 3 / REWARD -1 / Q_MAX  -13.806687 / Loss  1.0654644966125488
fps: 2.636626971355005
TIMESTEP 66109 / STATE explore / EPSILON 0.03405800799989925 / ACTION 3 / REWARD -1 / Q_MAX  -8.888119 / Loss  0.18312732875347137
fps: 2.6391885429513477
TIMESTEP 66110 / STATE explore / EPSILON 0.

fps: 1.9607311865958161
TIMESTEP 66156 / STATE explore / EPSILON 0.034011054999899176 / ACTION 4 / REWARD -0.1 / Q_MAX  -5.4253626 / Loss  0.06064055860042572
fps: 3.1903223254653548
TIMESTEP 66157 / STATE explore / EPSILON 0.034010055999899175 / ACTION 0 / REWARD -0.2 / Q_MAX  -5.2506223 / Loss  0.07773691415786743
fps: 3.219493024169778
TIMESTEP 66158 / STATE explore / EPSILON 0.03400905699989917 / ACTION 0 / REWARD -0.2 / Q_MAX  -3.2472107 / Loss  0.052026472985744476
fps: 3.0950846769730287
TIMESTEP 66159 / STATE explore / EPSILON 0.03400805799989917 / ACTION 0 / REWARD -0.2 / Q_MAX  -3.4244897 / Loss  0.041514988988637924
fps: 1.3969907257063967
TIMESTEP 66160 / STATE explore / EPSILON 0.03400705899989917 / ACTION 0 / REWARD -0.2 / Q_MAX  -0.39849234 / Loss  0.007205577101558447
fps: 3.229550167202191
TIMESTEP 66161 / STATE explore / EPSILON 0.03400605999989917 / ACTION 0 / REWARD -0.2 / Q_MAX  -5.6316876 / Loss  0.10846416652202606
fps: 1.3998492784657108
TIMESTEP 66162 / STATE e

TIMESTEP 66208 / STATE explore / EPSILON 0.0339591069998991 / ACTION 0 / REWARD -0.2 / Q_MAX  -7.827998 / Loss  0.006502226926386356
fps: 2.839607006293532
TIMESTEP 66209 / STATE explore / EPSILON 0.033958107999899095 / ACTION 0 / REWARD -0.2 / Q_MAX  -6.3579283 / Loss  0.009868440218269825
----------Random Action----------
fps: 1.8686797402393733
TIMESTEP 66210 / STATE explore / EPSILON 0.033957108999899094 / ACTION 3 / REWARD -0.1 / Q_MAX  -6.459277 / Loss  0.035921212285757065
----------Random Action----------
fps: 1.855453239243221
TIMESTEP 66211 / STATE explore / EPSILON 0.03395610999989909 / ACTION 1 / REWARD -0.1 / Q_MAX  -5.047042 / Loss  0.006058602128177881
fps: 1.9300599545267763
TIMESTEP 66212 / STATE explore / EPSILON 0.03395511099989909 / ACTION 4 / REWARD -0.1 / Q_MAX  -6.4405723 / Loss  0.07313515245914459
fps: 1.9354875030744518
TIMESTEP 66213 / STATE explore / EPSILON 0.03395411199989909 / ACTION 4 / REWARD -0.1 / Q_MAX  -5.096661 / Loss  0.02951238490641117
fps: 2.07

...

fps: 3.3274605874438326
TIMESTEP 66781 / STATE explore / EPSILON 0.03338667999989822 / ACTION 0 / REWARD -0.2 / Q_MAX  -14.819498 / Loss  0.02309073507785797
fps: 3.1863669237454495
TIMESTEP 66782 / STATE explore / EPSILON 0.03338568099989822 / ACTION 0 / REWARD -0.2 / Q_MAX  -18.35761 / Loss  0.021114680916070938
fps: 1.4482793864767995
TIMESTEP 66783 / STATE explore / EPSILON 0.03338468199989822 / ACTION 0 / REWARD -0.2 / Q_MAX  -7.1657915 / Loss  0.0038870074786245823
fps: 3.014312274203482
TIMESTEP 66784 / STATE explore / EPSILON 0.03338368299989822 / ACTION 0 / REWARD -0.2 / Q_MAX  -15.023239 / Loss  0.04340217262506485
fps: 3.014409760402697
TIMESTEP 66785 / STATE explore / EPSILON 0.033382683999898216 / ACTION 0 / REWARD -0.2 / Q_MAX  -7.624495 / Loss  0.04089915007352829
fps: 2.995948548277419
TIMESTEP 66786 / STATE explore / EPSILON 0.033381684999898215 / ACTION 0 / REWARD -0.2 / Q_MAX  -9.359797 / Loss  0.00858867634087801
fps: 2.3458627104216094
TIMESTEP 66787 / STATE explor

fps: 3.042386497251243
TIMESTEP 66833 / STATE explore / EPSILON 0.03333473199989814 / ACTION 0 / REWARD -0.2 / Q_MAX  -14.987008 / Loss  0.026627400889992714
fps: 3.0161417325665765
TIMESTEP 66834 / STATE explore / EPSILON 0.03333373299989814 / ACTION 0 / REWARD -0.2 / Q_MAX  -5.254136 / Loss  0.012796817347407341
fps: 3.036943176930752
TIMESTEP 66835 / STATE explore / EPSILON 0.03333273399989814 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.289354 / Loss  0.005904120858758688
fps: 2.8763532075802942
TIMESTEP 66836 / STATE explore / EPSILON 0.03333173499989814 / ACTION 0 / REWARD -0.2 / Q_MAX  -5.7918067 / Loss  0.01388717070221901
fps: 2.7282439292191047
TIMESTEP 66837 / STATE explore / EPSILON 0.03333073599989814 / ACTION 0 / REWARD -0.2 / Q_MAX  -8.345335 / Loss  0.0075295777060091496
fps: 2.7213421719719477
TIMESTEP 66838 / STATE explore / EPSILON 0.033329736999898135 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.740341 / Loss  0.01200318243354559
fps: 2.9863601443089185
TIMESTEP 66839 / STATE exp

fps: 1.7763194060052523
TIMESTEP 66885 / STATE explore / EPSILON 0.03328278399989806 / ACTION 0 / REWARD -0.2 / Q_MAX  -9.695398 / Loss  0.006516602821648121
fps: 1.7582391048637735
TIMESTEP 66886 / STATE explore / EPSILON 0.03328178499989806 / ACTION 0 / REWARD -0.2 / Q_MAX  -8.728398 / Loss  0.04230784624814987
fps: 1.83280554049206
TIMESTEP 66887 / STATE explore / EPSILON 0.03328078599989806 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.766999 / Loss  0.01673951931297779
fps: 1.803333905164582
TIMESTEP 66888 / STATE explore / EPSILON 0.03327978699989806 / ACTION 0 / REWARD -0.2 / Q_MAX  -16.633465 / Loss  0.015233282931149006
fps: 1.808320690233491
TIMESTEP 66889 / STATE explore / EPSILON 0.03327878799989806 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.214462 / Loss  0.009049706161022186
fps: 1.8098047718598775
TIMESTEP 66890 / STATE explore / EPSILON 0.033277788999898056 / ACTION 0 / REWARD -0.2 / Q_MAX  -13.434389 / Loss  0.022800985723733902
fps: 1.856824175391204
TIMESTEP 66891 / STATE explore

fps: 1.7651546523646475
TIMESTEP 66937 / STATE explore / EPSILON 0.033230835999897984 / ACTION 0 / REWARD -0.2 / Q_MAX  -15.45433 / Loss  0.014409925788640976
fps: 1.7979582607279188
TIMESTEP 66938 / STATE explore / EPSILON 0.03322983699989798 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.089932 / Loss  0.008456025272607803
fps: 1.8207897938975404
TIMESTEP 66939 / STATE explore / EPSILON 0.03322883799989798 / ACTION 0 / REWARD -0.2 / Q_MAX  -15.175579 / Loss  0.06163742020726204
fps: 1.8894171926304557
TIMESTEP 66940 / STATE explore / EPSILON 0.03322783899989798 / ACTION 0 / REWARD -0.2 / Q_MAX  -9.048022 / Loss  0.01150258257985115
fps: 1.816330896433338
TIMESTEP 66941 / STATE explore / EPSILON 0.03322683999989798 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.544815 / Loss  0.011051996611058712
fps: 1.8179400426669163
TIMESTEP 66942 / STATE explore / EPSILON 0.033225840999897976 / ACTION 0 / REWARD -0.2 / Q_MAX  -11.918414 / Loss  0.013342900201678276
fps: 1.798634440538299
TIMESTEP 66943 / STATE exp

WebDriverException: Message: chrome not reachable
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17763 x86_64)
